In [ ]:
import logging
import os
import sys
import time
from datetime import datetime, time
from typing import List, Tuple

from cachetools import TTLCache
from dotenv import load_dotenv

from instagrapi import Client
from instagrapi.exceptions import (
    ChallengeRequired,
    ClientError,
    PleaseWaitFewMinutes,
    TwoFactorRequired,
    BadPassword,
    LoginRequired,
    ReloginAttemptExceeded,
)
from instagrapi.types import UserShort
from services.instagram.instagrapi.extractors import extract_user_short
from services.instagram.instagrapi.mixins.user import MAX_USER_COUNT

# Load environment variables from .env file
load_dotenv()

# Verify presence of required environment variables
INSTA_USERNAME = os.getenv("INSTA_USERNAME")
INSTA_PASSWORD = os.getenv("INSTA_PASSWORD")
if not INSTA_USERNAME or not INSTA_PASSWORD:
    print("Error: INSTA_USERNAME or INSTA_PASSWORD not set in environment variables.")
    sys.exit(1)


def setup_logging():
    """Configure logging."""
    logging.basicConfig(level=os.getenv("LOG_LEVEL", "INFO").upper())
    logger_inst = logging.getLogger(__name__)  # Rename to logger_inst
    handler = logging.StreamHandler(sys.stdout)
    formatter = logging.Formatter("%(asctime)s - %(levelname)s - %(message)s")
    handler.setFormatter(formatter)
    logger_inst.addHandler(handler)
    return logger_inst


# Initialize cache for storing follower details temporarily
follower_info_cache = TTLCache(maxsize=1000, ttl=3600)

logger = setup_logging()


def validate_follower_data(follower_info, username):
    """Ensure follower data has necessary attributes and valid content."""
    if not follower_info:
        logger.error({"error": "Follower data is None.", "username": username})
        raise ValueError(f"Follower data for {username} is None.")
    required_attrs = ["username", "full_name", "pk"]
    missing_attrs = [attr for attr in required_attrs if not follower_info.get(attr)]
    if missing_attrs:
        logger.error(
            {
                "error": "Missing required attributes.",
                "username": username,
                "missing_attrs": missing_attrs,
            }
        )
        raise ValueError(f"Missing required attributes for {username}: {missing_attrs}")


def calculate_engagement_metrics(api_client, user_id):
    """Calculate engagement metrics based on the user's recent posts."""
    try:
        posts = api_client.user_feed(user_id, amount=10)
        total_likes = sum(post.like_count for post in posts)
        total_comments = sum(post.comment_count for post in posts)
        post_count = len(posts)
        avg_likes = total_likes / post_count if post_count else 0
        avg_comments = total_comments / post_count if post_count else 0
        engagement_ratio = (
            (total_likes + total_comments) / post_count if post_count else 0
        )
        return avg_likes, avg_comments, engagement_ratio
    except (ChallengeRequired, PleaseWaitFewMinutes, ClientError) as ex:
        logger.error(
            {"error": "Error calculating engagement metrics.", "exception": str(ex)}
        )
        return 0, 0, 0


class FetchError(Exception):
    """Custom exception class for fetch errors."""

    def __init__(self, message):
        self.message = message
        super().__init__(message)


def retry_on_error(retries=3, delay=1, backoff=2):
    def decorator(func):
        def wrapper(*args, **kwargs):
            current_delay = delay  # Initialize delay here
            for _ in range(retries):
                try:
                    return func(*args, **kwargs)
                except (ChallengeRequired, PleaseWaitFewMinutes, ClientError) as ex:
                    logger.warning(
                        {
                            "warning": "Retrying after recoverable error.",
                            "exception": str(ex),
                        }
                    )
                    time.sleep(current_delay)
                    current_delay *= backoff
            raise FetchError("Max retries exceeded. Error not recoverable.")

        return wrapper

    return decorator


@retry_on_error(retries=3, delay=1, backoff=2)
def fetch_follower_details(api_client, username):
    """Fetch detailed information for a specific follower."""
    cached_data = follower_info_cache.get(username)
    if cached_data:
        logger.debug(
            {"message": "Using cached data for follower.", "username": username}
        )
        return cached_data
    try:
        logger.debug(
            {"message": "Fetching details for follower.", "username": username}
        )
        follower_info = api_client.user_info_by_username(username)
        validate_follower_data(follower_info, username)
        metrics = calculate_engagement_metrics(api_client, follower_info.pk)
        follower_data = {
            "username": follower_info.username,
            "full_name": follower_info.full_name,
            "user_id": follower_info.pk,
            "average_likes_per_post": metrics[0],
            "average_comments_per_post": metrics[1],
            "engagement_ratio": metrics[2],
            "recent_activity": datetime.now().isoformat(),
        }
        follower_info_cache[username] = follower_data
        return follower_data
    except ClientError as ex:
        logger.error(
            {
                "error": "Error fetching details for follower.",
                "exception": str(ex),
                "username": username,
            }
        )
        return None
    except ValueError as ve:
        logger.error(
            {
                "error": "Error validating follower data.",
                "exception": str(ve),
                "username": username,
            }
        )
        return None
    except Exception as ex:
        logger.error(
            {
                "error": "Error fetching details for follower.",
                "exception": str(ex),
                "username": username,
            }
        )
        return None


def handle_login_exception(ex, api_client, username):
    """
    Handle exceptions that occur during login.
    """
    if isinstance(ex, BadPassword):
        if api_client.relogin_attempt > 0:
            api_client.freeze(str(ex), days=7)
            raise ReloginAttemptExceeded(ex)
        api_client.settings = api_client.rebuild_client_settings()
        return api_client.update_client_settings(api_client.get_settings())
    elif isinstance(ex, LoginRequired):
        logger.error(
            {
                "error": "Login required.",
                "exception": str(ex),
                "username": username,
            }
        )
        api_client.relogin()
        return api_client.update_client_settings(api_client.get_settings())
    else:
        logger.error(
            {
                "error": "Unexpected error during login.",
                "exception": str(ex),
                "username": username,
            }
        )
        return None


def login_user(username, password):
    """
    Log in the user using the provided credentials.
    """
    api_client = Client()
    try:
        api_client.login(username, password)
        logger.info({"message": "Login successful.", "username": username})
        return api_client
    except (TwoFactorRequired, BadPassword, LoginRequired) as ex:
        if isinstance(ex, TwoFactorRequired):
            logger.info(
                {"message": "Two-factor authentication required.", "username": username}
            )
        return handle_login_exception(ex, api_client, username)
    except Exception as e:
        logger.error(
            {
                "error": "Unexpected error during login.",
                "exception": str(e),
                "username": username,
            }
        )
        return None


def user_followers_v1(api_client, user_id: str, amount: int = 0) -> List[UserShort]:
    """
    Updated method to fetch user's followers, ensuring correct user ID usage and error handling.
    """
    users, _ = user_followers_v1_chunk(api_client, str(user_id), amount)
    if amount:
        users = users[:amount]
    return users


def user_followers_v1_chunk(
    api_client, user_id: str, max_amount: int = 0, max_id: str = ""
) -> Tuple[List[UserShort], bool]:
    """
    Fetch a chunk of user's followers with updated endpoint and error handling.
    """
    unique_set = set()
    users = []

    try:
        result = api_client.private_request(
            f"friendships/{user_id}/followers/",  # Updated endpoint
            params={
                "max_id": max_id,
                "count": max_amount or MAX_USER_COUNT,
                "rank_token": api_client.rank_token,
                "search_surface": "follow_list_page",
                "query": "",
                "enable_groups": "true",
            },
        )

        users.extend(extract_unique_users(result.get("users", []), unique_set))

        max_id = result.get("next_max_id")
        has_more = bool(max_id) and (not max_amount or len(users) < max_amount)
        return users, has_more

    except ClientError as e:
        if e.response.status_code == 404:
            logger.error({"error": "Endpoint not found.", "user_id": user_id})
        else:
            logger.error(
                {
                    "error": "Error fetching followers.",
                    "exception": str(e),
                    "user_id": user_id,
                }
            )

    return users, False


def extract_unique_users(users_data, unique_set):
    """
    Extract unique users from user data and update unique set.
    """
    unique_users = []
    for user in users_data:
        user = extract_user_short(user)
        if user.pk not in unique_set:
            unique_set.add(user.pk)
            unique_users.append(user)
    return unique_users


if __name__ == "__main__":
    client = login_user(INSTA_USERNAME, INSTA_PASSWORD)
    if client:
        logger.info(
            {"message": "Proceeding with main logic.", "username": INSTA_USERNAME}
        )
        followers = user_followers_v1(client, INSTA_USERNAME)
        for follower in followers:
            logger.info({"message": "Follower", "username": follower.username})
    else:
        logger.error({"error": "Login failed.", "username": INSTA_USERNAME})
